In [1]:
import shutil
import os
root_name = "./model3-sentinel2"
logs_dir = os.listdir(root_name)
for logs_name in logs_dir:
    log_files = os.listdir("%s/%s" % (root_name, logs_name))
    print(logs_name, log_files[-2])
    

order[001]-iter[150]-epoch[300]-rate[20.000]-lambda2[0.200]-rho[0.010, 1.020] iter[150]-pnsr[38.242, 42.476, 37.582, 36.953].png
order[002]-iter[150]-epoch[300]-rate[20.000]-lambda2[0.200]-rho[0.010, 1.030] iter[150]-pnsr[39.603, 47.299, 39.087, 37.638].png
order[003]-iter[150]-epoch[300]-rate[20.000]-lambda2[0.200]-rho[0.010, 1.040] iter[150]-pnsr[38.129, 42.140, 37.845, 37.062].png
order[004]-iter[150]-epoch[300]-rate[20.000]-lambda2[0.200]-rho[0.010, 1.050] iter[150]-pnsr[35.214, 38.181, 35.700, 34.659].png
order[005]-iter[150]-epoch[300]-rate[20.000]-lambda2[0.200]-rho[0.050, 1.020] iter[150]-pnsr[38.222, 42.448, 37.543, 36.929].png
order[006]-iter[150]-epoch[300]-rate[20.000]-lambda2[0.200]-rho[0.050, 1.030] iter[150]-pnsr[39.600, 47.297, 39.074, 37.623].png
order[007]-iter[150]-epoch[300]-rate[20.000]-lambda2[0.200]-rho[0.050, 1.040] iter[150]-pnsr[38.130, 42.157, 37.842, 37.051].png
order[008]-iter[150]-epoch[300]-rate[20.000]-lambda2[0.200]-rho[0.050, 1.050] iter[150]-pnsr[35.2

In [ ]:
from __future__ import print_function

import os
import mat73
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as scio
import torch
import torch.optim
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

from architecture import MST

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor
# dtype = torch.cuda.DoubleTensor
# dtype = torch.double
device = torch.device('cuda')
torch.cuda.manual_seed(seed=666)


########################################### 辅助函数 ###########################################
def thres_21(L, tau, M, N, B):
    S = torch.sqrt(torch.sum(torch.mul(L, L), 2))
    S[S == 0] = 1
    T = 1 - tau / S
    T[T < 0] = 0
    R = T.reshape(M, N, 1).repeat((1, 1, B))
    res = torch.mul(R, L)
    return res


################################ 读取数据 ################################
YY= mat73.loadmat("./Data/T31UDQ/T31UDQ_20m_raw1.mat")['T31UDQ_20m_raw1'] 
YY = np.double(YY)
YY = YY/YY.max()

Mask2_500 = mat73.loadmat("./Data/Mask2_500.mat")['Mask2_500']
Mask_Out_500 = mat73.loadmat("./Data/Mask_Out_500.mat")["Mask_Out_500"]
Mask_Shd = mat73.loadmat("./Data/Mask_Shd.mat")["Mask_Shd"]
Mask_shad = mat73.loadmat("./Data/Mask_shad.mat")["Mask_shad"]
Mask = np.copy(Mask2_500)
Mask[ Mask != 0 ] = 1
# Test1_Clean是干净图像，取的是4，2，0三个时相
Test1_Clean = np.copy(YY[:, :, :, [4, 2, 0]])

# 生成云图YY，含有6个时相
Clean = YY[:, :, :, [4, 2, 0, 5, 3, 1]]
YY = np.multiply((1 - Mask), (YY[:, :, :, [4, 2, 0, 5, 3, 1]])) + Mask
YY = np.multiply((1 - Mask_Out_500),  YY) + Mask_Out_500
YY = np.multiply(YY, (1 - Mask_Shd) ) - Mask_Shd
YY = np.multiply(YY, (1 - Mask_shad) ) - Mask_shad
Noisy = np.copy(YY)
Cloud = np.copy(Mask)
Mask = 1 - Cloud
for i in range(6):
    plt.subplot(3, 6, i+1)
    plt.imshow(Noisy[:, :, [4, 2, 1], i]*5)

    plt.subplot(3, 6, i+7)
    plt.imshow(Clean[:, :, [4, 2, 1], i]*5)

    plt.subplot(3, 6, i+13)
    plt.imshow(Mask[:, :, 0, i]*5)

plt.show()

In [ ]:
a = [1, 2, 3]
print("psnr%s" % a)

In [ ]:
for i in range(6):
    for j in range(6):
        plt.subplot(6, 6, i*6 + j+1)
        plt.imshow(Cloud[:, :, i, j]*5, cmap='gray')

plt.show()

In [ ]:
Mask = 1 - Cloud
for i in range(6):
    for j in range(6):
        plt.subplot(6, 6, i*6 + j+1)
        plt.imshow(Mask[:, :, i, j]*5, cmap='gray')

plt.show()

In [ ]:
res = Mask[:, :, :, 3:] == 1
res.all()

In [ ]:
from skimage.measure import compare_ssim, compare_psnr, compare_mse
import cv2

img1 = cv2.imread(r'C:\Users\Wxr\Desktop\WATERLOO\train_low_high\29.jpg')
img2 = cv2.imread(r'C:\Users\Wxr\Desktop\WATERLOO\train_normal\29.jpg')

psnr = compare_psnr(img1, img2)
ssim = compare_ssim(img1, img2, multichannel=True)  # 对于多通道图像(RGB、HSV等)关键词multichannel要设置为True
mse = compare_mse(img1, img2)

print('PSNR：{}，SSIM：{}，MSE：{}'.format(psnr, ssim, mse))


In [ ]:
from __future__ import print_function

import os
import mat73
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as scio
import torch
import torch.optim
from skimage.metrics import peak_signal_noise_ratio as compare_psnr
from skimage.metrics import structural_similarity as compare_ssim

from architecture import MST

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor
# dtype = torch.cuda.DoubleTensor
# dtype = torch.double
device = torch.device('cuda')
torch.cuda.manual_seed(seed=666)


########################################### 辅助函数 ###########################################
def thres_21(L, tau, M, N, B):
    S = torch.sqrt(torch.sum(torch.mul(L, L), 2))
    S[S == 0] = 1
    T = 1 - tau / S
    T[T < 0] = 0
    R = T.reshape(M, N, 1).repeat((1, 1, B))
    res = torch.mul(R, L)
    return res


################################ 读取数据 ################################
YY= mat73.loadmat("./Data/T31UDQ/T31UDQ_20m_raw1.mat")['T31UDQ_20m_raw1']

compare_ssim(YY[:, :, 0, 0], YY[:, :, 0, 1])